<a href="https://colab.research.google.com/github/Max-FM/icg_pakistan_innovation_workshop/blob/main/Downloading_Satellite_Images_From_Google_Earth_Engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Downloading Satellite Images From Google Earth Engine 

## Install geemap

[geemap](https://geemap.readthedocs.io/en/latest/readme.html) is a useful package that adds additional functionality to the Google Earth Engine Python API.

In [ ]:
%%capture

!pip install geemap

##Import required packages

In [ ]:
import ee
import glob
import folium
import os
import geemap.eefolium as emap

##Authenticate Google Earth Engine

To access the Google Earth Engine API you require an account. To request access, go to [https://signup.earthengine.google.com](https://signup.earthengine.google.com/). You may have to wait up to a day or so to be granted access and it's possible you will not recieve any email communication. To manually check whether you have access, try to log into [https://code.earthengine.google.com](https://code.earthengine.google.com/), or attempt to run the next cell and follow the instructions provided in the output cell.

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=yHvjYqv-IHjF00nSIZBIJ4q79chipjUeHP6q9eVqmBc&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWjyR1Dmf64xhB4X4pjLZDNO5tUY56Jbv7zQzMuJrLeWUOKtxGMjYdM

Successfully saved authorization token.


## Define Request Function

Defines a series of Google Earth Image Collections that we wish to download images from. Image collections are filtered to a input geographical region, date range and maxiumum allowed cloud coverage.

In [ ]:
def obtain_data(
    region, 
    start_date, 
    end_date,
    include_end_date=False, 
    max_cloud_cover=80
    ):
    start_date = ee.Date(start_date)

    if include_end_date:
        end_date = ee.Date(end_date)
    else:
        end_date = ee.Date(end_date).advance(-1, "day")
    
    # Filter input collections by desired date range, region and cloud coverage.
    criteria  = ee.Filter.And(
        ee.Filter.geometry(region), 
        ee.Filter.date(start_date, end_date)
    )
    
    Sentinel_2_SR = ee.ImageCollection('COPERNICUS/S2_SR') \
                      .filter(criteria) \
                      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', max_cloud_cover)) \
                      .select(['B2', 'B3', 'B4', 'B8', 'MSK_CLDPRB']) # B, G, R, NIR, Cloud Probability Map

    MODIS_16D_NDVI = ee.ImageCollection("MODIS/006/MOD13Q1") \
                       .filter(criteria) \
                       .select('NDVI')

    image_collections = {
        'Sentinel_2_SR': Sentinel_2_SR,
        'MODIS_16D_NDVI': MODIS_16D_NDVI
        }

    return image_collections

## Define Download Function

In [ ]:
def download_data(
    filepath, 
    region,
    scale_dict,
    start_date, 
    end_date,
    include_end_date=False,  
    max_cloud_cover=80, 
    desired_collections=None,
    overwrite=False,
    ):
    # Obtains all image collections defined in the request function for 
    # the chosen test district and date range, with maximum n% cloud cover.
    image_collections = obtain_data(
        region, 
        start_date, 
        end_date,
        include_end_date,
        max_cloud_cover
    )

    # Filters out unwanted collections if defined above.
    if desired_collections:
        image_collections = {
            collection: image_collections[collection] \
            for collection in desired_collections
        }    
    
    # Creates a subfolder in the base directory for the start date of the 
    # disaster.
    out_dir = f'{filepath}/{start_date}'
    if not os.path.isdir(out_dir):
        os.mkdir(out_dir)

    # Iterating through each image collection.
    for collection_name, collection in image_collections.items():
        print(collection_name)
        # Counts the number of images in a collection.
        collection_size = collection.size().getInfo()
        
        # Skips the image collection if it contains no images.
        if collection_size == 0:
            print('No images in collection, skipping.')
            continue
            
        # Creates additional subfolders for each image collection. 
        collection_dir = f'{out_dir}/{collection_name}'
        if not os.path.isdir(collection_dir):
            os.mkdir(collection_dir)

        # Counts number of .tif files already in image collection subfolder.
        tif_count = len(glob.glob1(collection_dir,"*.tif"))

        # Assumes the download for this collection is already complete and 
        # therefore skips, provided the number of .tif files already in 
        # chosen directory matches the number of images in the collection 
        # and overwrite is set to False.
        if collection_size == tif_count and overwrite == False:
            print('Correct number of .tif files for image collection already in directory, skipping.')
            continue

        # Exports each image in the filtered image collection to 
        # geoTIFF format.
        emap.ee_export_image_collection(
            collection, 
            collection_dir,
            crs='EPSG:4326',
            scale=scale_dict[collection_name],
            region=region
        )

## Initialising Parameters

In [ ]:
# Define geometry using lat, lon coordinates
# roi_coords = [71.63855314940405, 33.921892010457874, 71.77622557372045, 33.83124918682841]
roi_coords = [71.59538031310034,33.943608700041096, 71.76223517149877,33.8347393884192]
region = ee.Geometry.Rectangle(roi_coords)


# Defines the desired pixel scale for each image, have set to the native 
# resolution of each satellite.
scale_dict = {
    'Sentinel_2_SR': 10,
    'MODIS_16D_NDVI': 250
}

##Downloading Imaging for All Events

In [ ]:
years = ["2019", "2020", "2021"]
months = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]

date_list = [f"{year}-{month}-01" for year in years for month in months]
date_list.append("2022-01-01")

dates = [(date_list[i], date_list[i+1]) for i in range(len(date_list)-1)]

dates

[('2019-01-01', '2019-02-01'),
 ('2019-02-01', '2019-03-01'),
 ('2019-03-01', '2019-04-01'),
 ('2019-04-01', '2019-05-01'),
 ('2019-05-01', '2019-06-01'),
 ('2019-06-01', '2019-07-01'),
 ('2019-07-01', '2019-08-01'),
 ('2019-08-01', '2019-09-01'),
 ('2019-09-01', '2019-10-01'),
 ('2019-10-01', '2019-11-01'),
 ('2019-11-01', '2019-12-01'),
 ('2019-12-01', '2020-01-01'),
 ('2020-01-01', '2020-02-01'),
 ('2020-02-01', '2020-03-01'),
 ('2020-03-01', '2020-04-01'),
 ('2020-04-01', '2020-05-01'),
 ('2020-05-01', '2020-06-01'),
 ('2020-06-01', '2020-07-01'),
 ('2020-07-01', '2020-08-01'),
 ('2020-08-01', '2020-09-01'),
 ('2020-09-01', '2020-10-01'),
 ('2020-10-01', '2020-11-01'),
 ('2020-11-01', '2020-12-01'),
 ('2020-12-01', '2021-01-01'),
 ('2021-01-01', '2021-02-01'),
 ('2021-02-01', '2021-03-01'),
 ('2021-03-01', '2021-04-01'),
 ('2021-04-01', '2021-05-01'),
 ('2021-05-01', '2021-06-01'),
 ('2021-06-01', '2021-07-01'),
 ('2021-07-01', '2021-08-01'),
 ('2021-08-01', '2021-09-01'),
 ('2021-

In [ ]:
%%time

for start_date, end_date in dates:
    print(start_date, end_date)
    download_data(
        filepath="/content/drive/Shareddrives/ICG Data Analytics/Data/Pakistan/Shamshato", 
        region=region,
        scale_dict=scale_dict,
        start_date=start_date, 
        end_date=end_date,
        include_end_date=False,
        max_cloud_cover=50, 
        desired_collections=None,
        overwrite=True
    )

2019-01-01 2019-02-01
Sentinel_2_SR
Total number of images: 10

Exporting 1/10: 20190106T055221_20190106T055223_T42SYC.tif
Generating URL ...
Please wait ...
Data downloaded to /content/drive/Shareddrives/ICG Data Analytics/Data/Pakistan/Shamshato/2019-01-01/Sentinel_2_SR/20190106T055221_20190106T055223_T42SYC.tif


Exporting 2/10: 20190106T055221_20190106T055223_T43SBT.tif
Generating URL ...
Please wait ...
Data downloaded to /content/drive/Shareddrives/ICG Data Analytics/Data/Pakistan/Shamshato/2019-01-01/Sentinel_2_SR/20190106T055221_20190106T055223_T43SBT.tif


Exporting 3/10: 20190109T060221_20190109T060220_T42SYC.tif
Generating URL ...
Please wait ...
Data downloaded to /content/drive/Shareddrives/ICG Data Analytics/Data/Pakistan/Shamshato/2019-01-01/Sentinel_2_SR/20190109T060221_20190109T060220_T42SYC.tif


Exporting 4/10: 20190109T060221_20190109T060220_T43SBT.tif
Generating URL ...
Please wait ...
Data downloaded to /content/drive/Shareddrives/ICG Data Analytics/Data/Pakistan/